In [1]:
import cv2
import numpy as np

In [2]:
def compute_gradients(image):
    # Compute gradients in x and y directions using Sobel operator
    dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    return dx, dy

In [3]:
def compute_integral_image(image):
    # Compute integral image for fast box filter computation
    integral_image = np.cumsum(np.cumsum(image, axis=0), axis=1)
    return integral_image

In [4]:
def compute_box_filter(integral_image, x, y, width, height):
    # Compute sum of pixels in the given box using integral image
    x0, y0 = max(x - width // 2, 0), max(y - height // 2, 0)
    x1, y1 = min(x + width // 2, integral_image.shape[1] - 1), min(y + height // 2, integral_image.shape[0] - 1)
    
    upper_left = integral_image[y0, x0]
    bottom_right = integral_image[y1, x1]
    upper_right = integral_image[y0, x1]
    bottom_left = integral_image[y1, x0]
    
    return bottom_right - upper_right - bottom_left + upper_left

In [5]:
def detect_surf(image, threshold=1000):
    # Compute gradients
    dx, dy = compute_gradients(image)
    
    # Compute integral image
    integral_image = compute_integral_image(image)
    
    keypoints = []
    
    # Define parameters
    num_octaves = 4
    num_scales = 5
    k = 2 ** (1 / num_scales)
    initial_sigma = 1.6
    
    # Generate scale space
    for octave in range(num_octaves):
        sigma = initial_sigma
        
        for scale in range(num_scales):
            # Compute scale factor
            factor = (2 ** octave) * k ** scale
            scale_dx = cv2.resize(dx, None, fx=factor, fy=factor)
            scale_dy = cv2.resize(dy, None, fx=factor, fy=factor)
            
            # Compute Harris response
            hessian_xx = cv2.GaussianBlur(scale_dx * scale_dx, (9, 9), sigma)
            hessian_xy = cv2.GaussianBlur(scale_dx * scale_dy, (9, 9), sigma)
            hessian_yy = cv2.GaussianBlur(scale_dy * scale_dy, (9, 9), sigma)
            harris_response = (hessian_xx * hessian_yy - hessian_xy ** 2) - 0.04 * ((hessian_xx + hessian_yy) ** 2)
            
            # Non-maximum suppression
            local_maxima = (harris_response == cv2.dilate(harris_response, np.ones((3, 3))))
            keypoints.extend(zip(*np.where(local_maxima & (harris_response > threshold))))
            
            # Update sigma for next scale
            sigma *= k
    
    return keypoints